# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
BASE_DIR = "/content/drive/MyDrive/stat_arb_pairs"


#1. Install & Import Libraries

In [3]:
!pip install yfinance pandas numpy --quiet


In [4]:
import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf


#2. Config: Universe, Paths and Dates

In [5]:
# 1) Asset universe (you can edit this later)
UNIVERSE = [
    # Index ETFs
    "SPY", "QQQ", "DIA",
    # Sector ETFs
    "XLF", "XLE", "XLK", "XLI", "XLP",
    # Large-cap stocks (good pairs candidates)
    "AAPL", "MSFT", "JPM", "BAC", "KO", "PEP", "XOM", "CVX",
]

# 2) Date range (last 5 years)
END_DATE = datetime.today()
START_DATE = END_DATE - timedelta(days=5 * 365)

# 3) Data folders (inside BASE_DIR defined earlier)
DATA_DIR_RAW = os.path.join(BASE_DIR, "data", "raw")
DATA_DIR_CLEANED = os.path.join(BASE_DIR, "data", "cleaned")

os.makedirs(DATA_DIR_RAW, exist_ok=True)
os.makedirs(DATA_DIR_CLEANED, exist_ok=True)

print("Base dir:", BASE_DIR)
print("Saving raw data to:", DATA_DIR_RAW)
print("Saving cleaned data to:", DATA_DIR_CLEANED)


Base dir: /content/drive/MyDrive/stat_arb_pairs
Saving raw data to: /content/drive/MyDrive/stat_arb_pairs/data/raw
Saving cleaned data to: /content/drive/MyDrive/stat_arb_pairs/data/cleaned


#3.Download Function

In [6]:
def download_price_data(tickers, start_date, end_date):
    """
    Download daily OHLCV data for a list of tickers using yfinance.
    Returns a long-format DataFrame: [date, ticker, open, high, low, close, adj_close, volume]
    """
    print(f"Downloading data for {len(tickers)} tickers from Yahoo Finance...")
    data = yf.download(
        tickers=tickers,
        start=start_date.strftime("%Y-%m-%d"),
        end=end_date.strftime("%Y-%m-%d"),
        auto_adjust=False,
        group_by="ticker",
        progress=True,
    )

    all_rows = []

    for ticker in tickers:
        if ticker not in data.columns.get_level_values(0):
            print(f"WARNING: No data for {ticker}, skipping.")
            continue

        df_t = data[ticker].copy()
        df_t["ticker"] = ticker
        df_t.rename(
            columns={
                "Open": "open",
                "High": "high",
                "Low": "low",
                "Close": "close",
                "Adj Close": "adj_close",
                "Volume": "volume",
            },
            inplace=True,
        )
        df_t.index.name = "date"
        all_rows.append(df_t.reset_index())

    if not all_rows:
        raise ValueError("No data downloaded for any ticker. Check ticker symbols or connection.")

    df_all = pd.concat(all_rows, axis=0, ignore_index=True)
    df_all.sort_values(["date", "ticker"], inplace=True)

    return df_all


#4. Cleaning + Aligning Function

In [7]:
def clean_and_align_data(df):
    """
    Clean the raw long-format DataFrame:
    - Keep adj_close only (for now)
    - Pivot to wide format with dates as index and tickers as columns
    - Forward-fill/back-fill gaps
    - Drop rows with any remaining NaNs
    - Compute log returns
    """
    print("Cleaning and aligning data...")

    df["date"] = pd.to_datetime(df["date"])
    df.sort_values(["date", "ticker"], inplace=True)

    # Wide format prices
    price_wide = df.pivot(index="date", columns="ticker", values="adj_close")

    # Fill gaps
    price_wide_ffill = price_wide.ffill().bfill()

    # Keep only rows where all tickers have data
    price_wide_cleaned = price_wide_ffill.dropna(how="any")

    # Log returns
    log_returns = np.log(price_wide_cleaned / price_wide_cleaned.shift(1))
    log_returns = log_returns.dropna(how="any")

    return price_wide_cleaned, log_returns


#5. Save to CSV

In [8]:
def save_data(raw_df, price_wide, log_returns):
    raw_path_csv = os.path.join(DATA_DIR_RAW, "prices_raw.csv")
    prices_csv = os.path.join(DATA_DIR_CLEANED, "adj_close_wide.csv")
    returns_csv = os.path.join(DATA_DIR_CLEANED, "log_returns_wide.csv")

    print(f"Saving raw data to: {raw_path_csv}")
    raw_df.to_csv(raw_path_csv, index=False)

    print(f"Saving cleaned price data to: {prices_csv}")
    price_wide.to_csv(prices_csv)

    print(f"Saving log returns to: {returns_csv}")
    log_returns.to_csv(returns_csv)


#6. Running Pipeline

In [9]:
print("=== STAT ARB DATA PIPELINE: STEP 1 (DOWNLOAD + CLEAN) ===")
print(f"Universe: {UNIVERSE}")
print(f"Date Range: {START_DATE.date()} to {END_DATE.date()}")

raw_df = download_price_data(UNIVERSE, START_DATE, END_DATE)
print("Downloaded rows:", len(raw_df))

prices_wide, log_returns = clean_and_align_data(raw_df)
print("Cleaned prices shape:", prices_wide.shape)
print("Log returns shape:", log_returns.shape)

save_data(raw_df, prices_wide, log_returns)

print("Done")


=== STAT ARB DATA PIPELINE: STEP 1 (DOWNLOAD + CLEAN) ===
Universe: ['SPY', 'QQQ', 'DIA', 'XLF', 'XLE', 'XLK', 'XLI', 'XLP', 'AAPL', 'MSFT', 'JPM', 'BAC', 'KO', 'PEP', 'XOM', 'CVX']
Date Range: 2020-12-05 to 2025-12-04


[*********************100%***********************]  16 of 16 completed


Downloaded rows: 20064
Cleaning and aligning data...
Cleaned prices shape: (1254, 16)
Log returns shape: (1253, 16)
Saving raw data to: /content/drive/MyDrive/stat_arb_pairs/data/raw/prices_raw.csv
Saving cleaned price data to: /content/drive/MyDrive/stat_arb_pairs/data/cleaned/adj_close_wide.csv
Saving log returns to: /content/drive/MyDrive/stat_arb_pairs/data/cleaned/log_returns_wide.csv
Done
